In [1]:
import pyspark

try: 
    type(sc)
except NameError:
    sc = pyspark.SparkContext('local[*]')


### 2016-01 Parcial
UBER almacena en un cluster todos los datos sobre el movimiento y viajes de todos sus vehículos. Existe un proceso que nos devuelve un RDD llamado trip_summary con los siguientes campos: (driver_id, car_id, trip_id, customer_id, date (YYYYMMDD), distance_traveled), Programar usando Py Spark un programa que nos indique cual fue el conductor con mayor promedio de distancia recorrida por viaje para Abril de 2016.



In [2]:
trips = [
    (1, 1, 1, 1, '20160101', 10),
    (2, 2, 2, 2, '20160202', 20),
    (1, 1, 3, 1, '20160402', 15),
    (1, 1, 4, 3, '20160405', 20),
    (2, 2, 5, 4, '20160410', 25),
    (3, 3, 6, 3, '20160415', 15),
    (2, 2, 7, 1, '20160420', 40),
    (3, 3, 8, 2, '20160505', 80)
]

In [4]:
data = sc.parallelize(trips)

In [5]:
data.filter(lambda x: x[4] < '20160501' and x[4] > '20160331')\
.map(lambda x: (x[0], (1, x[5]))).collect()

#para usar reduceByKey primero tengo que tener una tupla (clave, valor)
#hasta aqui la forma del rdd es (id_conductor, (1, kilom_realizados_viaje_xi) )
#como tenenmos con cada id un 1 en la segunda posicion de la tupla podemos contar la cantidad total
#de viajes que hizo el conductor y el kilometraje total usando reducebykey
                          

[(1, (1, 15)), (1, (1, 20)), (2, (1, 25)), (3, (1, 15)), (2, (1, 40))]

In [6]:
data.filter(lambda x: x[4] > '20160400' and x[4] < '20160500')\
    .map(lambda x: (x[0], (1, x[5])))\
    .reduceByKey(lambda x,y: (x[1]+y[1],x[0]+y[0])).collect() #fijemonos que si corremos así el reducebykey
#el conductor 3 -como aparece una sola vez en los registros- no se reduce con nadie y por ende queda como estaba (OJO!!!)

#la conclusion es mantener el orden incial de la tupla para que esto no pase

[(1, (35, 2)), (2, (65, 2)), (3, (1, 15))]

In [7]:
data.filter(lambda x: x[4] > '20160400' and x[4] < '20160500')\
    .map(lambda x: (x[0], (1, x[5])))\
    .reduceByKey(lambda x,y: (x[0]+y[0],x[1]+y[1])).collect()

[(1, (2, 35)), (2, (2, 65)), (3, (1, 15))]

In [8]:
resul_tupla = data.filter(lambda x: x[4] < '20160501' and x[4] > '20160331')\
.map(lambda x: (x[0], (1, x[5])))\
.reduceByKey(lambda x,y: (x[0] + y[0], x[1] + y[1]))\
.reduce(lambda x,y: x if x[1][1]/x[1][0] > y[1][1]/y[1][0] else y)

In [9]:
resul_tupla

(2, (2, 65))